In [ ]:
!pip install ipython-autotime
%load_ext autotime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 3.24 s (started: 2022-12-07 10:36:30 +00:00)


In [ ]:
import pandas as pd
import numpy as np
import random

time: 608 µs (started: 2022-12-07 10:36:33 +00:00)


In [ ]:
%cd /content/drive/My Drive/Assignment-2/
df = pd.read_csv("train.csv")
df = df.iloc[:1000]
df

/content/drive/My Drive/Assignment-2


,policy_id,policy_tenure,age_of_car,age_of_policyholder,area_cluster,population_density,make,segment,model,fuel_type,...,is_brake_assist,is_power_door_locks,is_central_locking,is_power_steering,is_driver_seat_height_adjustable,is_day_night_rear_view_mirror,is_ecw,is_speed_alert,ncap_rating,is_claim
0,ID00001,0.515874,0.05,0.644231,C1,4990,1,A,M1,CNG,...,No,No,No,Yes,No,No,No,Yes,0,0
1,ID00002,0.672619,0.02,0.375000,C2,27003,1,A,M1,CNG,...,No,No,No,Yes,No,No,No,Yes,0,0
2,ID00003,0.841110,0.02,0.384615,C3,4076,1,A,M1,CNG,...,No,No,No,Yes,No,No,No,Yes,0,0
3,ID00004,0.900277,0.11,0.432692,C4,21622,1,C1,M2,Petrol,...,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,2,0
4,ID00005,0.596403,0.11,0.634615,C5,34738,2,A,M3,Petrol,...,No,Yes,Yes,Yes,No,Yes,Yes,Yes,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,ID00996,0.483288,0.11,0.413462,C5,34738,3,C2,M4,Diesel,...,Yes,Yes,Yes,Yes,Yes,No,Yes,Yes,3,0
996,ID00997,0.214615,0.18,0.519231,C8,8794,1,B2,M6,Petrol,...,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,2,0
997,ID00998,1.199612,0.13,0.701923,C5,34738,1,B2,M6,Petrol,...,Yes,Yes,Yes,Yes,Yes,Yes,Yes,Yes,2,1
998,ID00999,0.147596,0.02,0.375000,C3,4076,1,A,M1,CNG,...,No,No,No,Yes,No,No,No,Yes,0,0


time: 368 ms (started: 2022-12-07 10:36:33 +00:00)


In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=.2, random_state=41)

X_train = train.iloc[:,:-1]
Y_train = train.iloc[:,-1].values
X_test = test.iloc[:,:-1]
Y_test = test.iloc[:,-1].values

time: 6.21 ms (started: 2022-12-07 10:36:34 +00:00)


# Random Forest Classifier
##### Reference: https://carbonati.github.io/posts/random-forests-from-scratch/

In [ ]:
# entropy takes in a probability of a class within a node.
def entropy(p):
    if p == 0:
        return 0
    elif p == 1:
        return 0
    else:
        return - (p * np.log2(p) + (1 - p) * np.log2(1-p))

# information_gain takes in a list of the classes from the left and right child and returns the information gain of that particular split.
def information_gain(left_child, right_child):
    parent = left_child + right_child
    p_parent = parent.count(1) / len(parent) if len(parent) > 0 else 0
    p_left = left_child.count(1) / len(left_child) if len(left_child) > 0 else 0
    p_right = right_child.count(1) / len(right_child) if len(right_child) > 0 else 0
    IG_p = entropy(p_parent)
    IG_l = entropy(p_left)
    IG_r = entropy(p_right)
    return IG_p - len(left_child) / len(parent) * IG_l - len(right_child) / len(parent) * IG_r

time: 1.83 ms (started: 2022-12-07 10:36:34 +00:00)


In [ ]:
# draw_bootstrap function can take in the training input X in the form of a dataframe and also the output y in the form of an array.
# It also returns the bootstrap sampled X_boot and y_boot that will be used to construct a tree
# and returns the out-of-bag observations that were left out for training.
def draw_bootstrap(X_train, y_train):
    bootstrap_indices = list(np.random.choice(range(len(X_train)), len(X_train), replace = True))
    oob_indices = [i for i in range(len(X_train)) if i not in bootstrap_indices]
    X_bootstrap = X_train.iloc[bootstrap_indices].values
    y_bootstrap = y_train[bootstrap_indices]
    X_oob = X_train.iloc[oob_indices].values
    y_oob = y_train[oob_indices]
    return X_bootstrap, y_bootstrap, X_oob, y_oob

def oob_score(tree, X_test, y_test):
    mis_label = 0
    for i in range(len(X_test)):
        pred = predict_tree(tree, X_test[i])
        if pred != y_test[i]:
            mis_label += 1
    return mis_label / len(X_test)

time: 2.34 ms (started: 2022-12-07 10:36:34 +00:00)


In [ ]:
"""
find_split_point does the following:
  1. select m features at random
  2. for each feature selected, iterate through each value in the bootstrapped dataset and compute the information gain
  3. Return a dictionary from the value that gives the highest information gain, which will represent a node in our tree consisting of:
      (int) the feature index
      (int) the value to split at
      (dictionary) left child node
      (dictionary) right child node
"""

def find_split_point(X_bootstrap, y_bootstrap, max_features):
    feature_ls = list()
    num_features = len(X_bootstrap[0])

    while len(feature_ls) <= max_features:
      feature_idx = random.sample(range(num_features), 1)
      if feature_idx not in feature_ls:
        feature_ls.extend(feature_idx)

    best_info_gain = -999
    node = None
    for feature_idx in feature_ls:
      for split_point in X_bootstrap[:,feature_idx]:
          left_child = {'X_bootstrap': [], 'y_bootstrap': []}
          right_child = {'X_bootstrap': [], 'y_bootstrap': []}

          # split children for continuous variables
          if type(split_point) in [int, float]:
              for i, value in enumerate(X_bootstrap[:,feature_idx]):
                  if value <= split_point:
                      left_child['X_bootstrap'].append(X_bootstrap[i])
                      left_child['y_bootstrap'].append(y_bootstrap[i])
                  else:
                      right_child['X_bootstrap'].append(X_bootstrap[i])
                      right_child['y_bootstrap'].append(y_bootstrap[i])
          # split children for categoric variables
          else:
              for i, value in enumerate(X_bootstrap[:,feature_idx]):
                  if value == split_point:
                      left_child['X_bootstrap'].append(X_bootstrap[i])
                      left_child['y_bootstrap'].append(y_bootstrap[i])
                  else:
                      right_child['X_bootstrap'].append(X_bootstrap[i])
                      right_child['y_bootstrap'].append(y_bootstrap[i])

          split_info_gain = information_gain(left_child['y_bootstrap'], right_child['y_bootstrap'])
          if split_info_gain > best_info_gain:
              best_info_gain = split_info_gain
              left_child['X_bootstrap'] = np.array(left_child['X_bootstrap'])
              right_child['X_bootstrap'] = np.array(right_child['X_bootstrap'])
              node = {'information_gain': split_info_gain,
                      'left_child': left_child,
                      'right_child': right_child,
                      'split_point': split_point,
                      'feature_idx': feature_idx}


    return node

time: 3.1 ms (started: 2022-12-07 10:36:34 +00:00)


In [ ]:
def terminal_node(node):
    y_bootstrap = node['y_bootstrap']
    pred = max(y_bootstrap, key = y_bootstrap.count)
    return pred

"""
split_node works as follows:
  1. Given a node, store the left and right children as left_child & right_child and remove them from the original dictionary
  2. Check if either children has 0 observations in them. If one of the children is entirely empty this ultimately means that the best split in the data for that node was unable to differentiate the 2 classes and its best to call terminal_node and return the tree. terminal_node returns the class with the highest counts at the current node.
  3. Check if the current depth of the tree has reached the maximum depth. If so, create a terminal node and return the tree
  4. Check if number of observations in the left child at the current node is less than the minimum samples needed to make a split, which will be stored as min_samples_split. If so create a terminal node and return the tree
  5. If the left child has more observations than min_samples_split we'll feed that left node into find_split_point again to find the best split point and repeat steps 1 - 6. This is ultimately going to be nesting dictionaries, which we are using to represent each node in our tree.
  6. Repeat steps 4 and 5 for the right child node
  7. Repeat steps 1 - 6 until each branch has a terminal node
"""


def split_node(node, max_features, min_samples_split, max_depth, depth):
    left_child = node['left_child']
    right_child = node['right_child']

    del(node['left_child'])
    del(node['right_child'])

    if len(left_child['y_bootstrap']) == 0 or len(right_child['y_bootstrap']) == 0:
        empty_child = {'y_bootstrap': left_child['y_bootstrap'] + right_child['y_bootstrap']}
        node['left_split'] = terminal_node(empty_child)
        node['right_split'] = terminal_node(empty_child)
        return

    if depth >= max_depth:
        node['left_split'] = terminal_node(left_child)
        node['right_split'] = terminal_node(right_child)
        return node

    if len(left_child['X_bootstrap']) <= min_samples_split:
        node['left_split'] = node['right_split'] = terminal_node(left_child)
    else:
        node['left_split'] = find_split_point(left_child['X_bootstrap'], left_child['y_bootstrap'], max_features)
        split_node(node['left_split'], max_depth, min_samples_split, max_depth, depth + 1)
    if len(right_child['X_bootstrap']) <= min_samples_split:
        node['right_split'] = node['left_split'] = terminal_node(right_child)
    else:
        node['right_split'] = find_split_point(right_child['X_bootstrap'], right_child['y_bootstrap'], max_features)
        split_node(node['right_split'], max_features, min_samples_split, max_depth, depth + 1)

time: 4.66 ms (started: 2022-12-07 10:36:34 +00:00)


In [ ]:
def build_tree(X_bootstrap, y_bootstrap, max_depth, min_samples_split, max_features):
    root_node = find_split_point(X_bootstrap, y_bootstrap, max_features)
    split_node(root_node, max_features, min_samples_split, max_depth, 1)
    return root_node

def random_forest(X_train, y_train, n_estimators, max_features, max_depth, min_samples_split):
    tree_ls = list()
    oob_ls = list()
    for i in range(n_estimators):
        X_bootstrap, y_bootstrap, X_oob, y_oob = draw_bootstrap(X_train, y_train)
        tree = build_tree(X_bootstrap, y_bootstrap, max_features, max_depth, min_samples_split)
        tree_ls.append(tree)
        oob_error = oob_score(tree, X_oob, y_oob)
        oob_ls.append(oob_error)
    print("OOB estimate: {:.2f}".format(np.mean(oob_ls)))
    return tree_ls

time: 1.85 ms (started: 2022-12-07 10:36:34 +00:00)


In [ ]:
def predict_tree(tree, X_test):
    feature_idx = tree['feature_idx']

    if X_test[feature_idx] <= tree['split_point']:
        if type(tree['left_split']) == dict:
            return predict_tree(tree['left_split'], X_test)
        else:
            value = tree['left_split']
            return value
    else:
        if type(tree['right_split']) == dict:
            return predict_tree(tree['right_split'], X_test)
        else:
            return tree['right_split']

time: 1.14 ms (started: 2022-12-07 10:36:34 +00:00)


In [ ]:
def predict_rf(tree_ls, X_test):
    pred_ls = list()
    for i in range(len(X_test)):
        ensemble_preds = [predict_tree(tree, X_test.values[i]) for tree in tree_ls]
        final_pred = max(ensemble_preds, key = ensemble_preds.count)
        pred_ls.append(final_pred)
    return np.array(pred_ls)

time: 1.19 ms (started: 2022-12-07 10:36:34 +00:00)


In [ ]:
model = random_forest(X_train, Y_train, n_estimators=100, max_features=3, max_depth=10, min_samples_split=2)

preds = predict_rf(model, X_test)
acc = sum(preds == Y_test) / len(Y_test)
print("Testing accuracy: {}".format(acc))

OOB estimate: 0.23
Testing accuracy: 0.925
time: 5min 55s (started: 2022-12-07 10:36:34 +00:00)


# K-Fold Cross Validation

In [ ]:
kfolds_data = df.drop(columns=['is_claim'])
kfolds_target = df["is_claim"]

time: 4.46 ms (started: 2022-12-07 10:42:30 +00:00)


In [ ]:
from sklearn.model_selection import KFold

time: 630 µs (started: 2022-12-07 10:42:30 +00:00)


## k = 3

In [ ]:
folds_k3 = KFold(n_splits=3)

scores_rf_k3 = []

for train_index, test_index in folds_k3.split(kfolds_data, kfolds_target):
    X_train = kfolds_data.iloc[train_index]
    Y_train = kfolds_target.iloc[train_index].values
    X_test = kfolds_data.iloc[test_index]
    Y_test = kfolds_target.iloc[test_index].values
    
    model = random_forest(X_train, Y_train, n_estimators=100, max_features=3, max_depth=10, min_samples_split=2)
    preds = predict_rf(model, X_test)
    acc = sum(preds == Y_test) / len(Y_test)
    scores_rf_k3.append(acc)

scores_rf_k3 = np.array(scores_rf_k3)
scores_rf_k3.mean()

OOB estimate: 0.21
OOB estimate: 0.21
OOB estimate: 0.17


0.9299748850647055

time: 13min 5s (started: 2022-12-07 10:42:30 +00:00)


## k = 5

In [ ]:
folds_k5 = KFold(n_splits=5)

scores_rf_k5 = []

for train_index, test_index in folds_k5.split(kfolds_data, kfolds_target):
    X_train = kfolds_data.iloc[train_index]
    Y_train = kfolds_target.iloc[train_index].values
    X_test = kfolds_data.iloc[test_index]
    Y_test = kfolds_target.iloc[test_index].values
    
    model = random_forest(X_train, Y_train, n_estimators=100, max_features=3, max_depth=10, min_samples_split=2)
    preds = predict_rf(model, X_test)
    acc = sum(preds == Y_test) / len(Y_test)
    scores_rf_k5.append(acc)

scores_rf_k5 = np.array(scores_rf_k5)
scores_rf_k5.mean()

OOB estimate: 0.17
OOB estimate: 0.21
OOB estimate: 0.17
OOB estimate: 0.18
OOB estimate: 0.20


0.93

time: 29min 46s (started: 2022-12-07 10:55:35 +00:00)


## k = 10

In [ ]:
folds_k10 = KFold(n_splits=10)

scores_rf_k10 = []

for train_index, test_index in folds_k10.split(kfolds_data, kfolds_target):
    X_train = kfolds_data.iloc[train_index]
    Y_train = kfolds_target.iloc[train_index].values
    X_test = kfolds_data.iloc[test_index]
    Y_test = kfolds_target.iloc[test_index].values
    
    model = random_forest(X_train, Y_train, n_estimators=100, max_features=3, max_depth=10, min_samples_split=2)
    preds = predict_rf(model, X_test)
    acc = sum(preds == Y_test) / len(Y_test)
    scores_rf_k10.append(acc)

scores_rf_k10 = np.array(scores_rf_k10)
scores_rf_k10.mean()

OOB estimate: 0.17
OOB estimate: 0.18
OOB estimate: 0.17
OOB estimate: 0.23
OOB estimate: 0.20
OOB estimate: 0.19
OOB estimate: 0.19
OOB estimate: 0.20
OOB estimate: 0.20
OOB estimate: 0.16


0.93

time: 1h 16min 36s (started: 2022-12-07 11:25:22 +00:00)
